In [2]:
from PIL import Image, ImageEnhance
import numpy as np 
import os
import cv2
from skimage.color import rgb2lab, lab2rgb
from skimage import color
from skimage.io import imsave
# np.save('example_1', a)
# b = np.load('example/example_1.npy')
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, ReLU
from keras.layers import Activation, Dense, Dropout, Flatten
#from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import load_model
import pandas as pd 
# import tensorflow as tf
# from keras.models import load_model
# import keras 

In [3]:
# image_folder = "/Users/alexeyvaganov/doc/files/color_matcher/type"
image_massiv = "/Users/alexeyvaganov/doc/files/color_matcher/temp"

In [4]:
pixel = 1

# Обучение

In [5]:
def show_image(img_file):
    img = Image.open(img_file)
    img.show()
# array_transform_nastr = [
#     [1,1,0.8],
#     [1,1,2]
# ]
# array_transform_nastr = [
#     [1.5,0.5,0.5],
#     [1.5,1,0.5],
#     [1,1,0.8],
#     [1,1,2],
#     [2,1.2,0],
#     [2,1.2,0.1]
# ]

array_transform_nastr = [
    [1.5,0.5,0.5],
    [1.5,1,0.5],
    [1,1,2]
]

In [7]:
def transform_image(img_pil,brightness,contrast,color):
    if (brightness != 0):
        enhancer = ImageEnhance.Brightness(img_pil)
        img_pil = enhancer.enhance(brightness)
    if (contrast != 0):
        enhancer = ImageEnhance.Contrast(img_pil)
        img_pil = enhancer.enhance(contrast)
    if (color != 0):
        enhancer = ImageEnhance.Color(img_pil)
        img_pil = enhancer.enhance(color) 
    return img_pil

In [8]:
def crop_image(array_x_y,a,b):
    h=0
    while h <= a.shape[0]-pixel:
        w=0
        while w <= a.shape[1]-pixel:
            #print (h,",",h+56)
            array_x_y.append([a[h,w][1],a[h,w][2],b[h,w][1],b[h,w][2]])
#             x_ = a[h,w]
#             y_ = b[h,w]
#             array_x_y.append([x_,y_])
            w+=pixel
        h+=pixel

# Из одной папки

In [9]:
image_folder = "/Users/alexeyvaganov/doc/files/color_matcher/slide_X_2"
array_image = []
for root, dirs, files in os.walk(image_folder):
    for file in files:
        if file == ".DS_Store":
            continue
        img_pil = Image.open(root + "/" + file)
        img_pil = img_pil.convert('RGB')
        img_pil_lab = rgb2lab(img_pil)
        #img_cv2_ish = cv2.imread(root + "/" + file)
        for x in array_transform_nastr:
            img_ = transform_image(img_pil,x[0],x[1],x[2])
            #img_.save("/Users/alexeyvaganov/doc/files/color_matcher/temp/geeks.jpg")
            #img_cv2_transf = cv2.imread("/Users/alexeyvaganov/doc/files/color_matcher/temp/geeks.jpg")
            img_lab = rgb2lab(img_)
            #img_pil = rgb2lab(img_pil)
            array_image.append([np.array(img_lab),np.array(img_pil_lab)])
        
array_full_image = np.array(array_image) 
array_full_image.shape

/var/folders/87/c02w9bp14k1fx5943h90k0500000gn/T/ipykernel_83530/1632532704.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array_full_image = np.array(array_image)


(54, 2)

In [56]:
imsave("/Users/alexeyvaganov/doc/files/color_matcher/temp/tttt.jpg", lab2rgb(array_full_image[0,1]))


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


# Подготовка данных

In [10]:
data = []
i=0
x_sgape = array_full_image.shape[0]
while i < x_sgape:
    crop_image(data,array_full_image[i,0],array_full_image[i,1])
    i+=1

dFrame = pd.DataFrame(data) 

# array_x_y = []
# i=0
# x_sgape = array_full_image.shape[0]
# while i < x_sgape:
#     crop_image(array_x_y,array_full_image[i,0],array_full_image[i,1])
#     i+=1
# array_crop_image = np.array(array_x_y) 
# array_crop_image = 1.0/255*array_crop_image
# array_crop_image.shape

In [11]:
dFrame

,0,1,2,3
0,-2.228665,1.182758,-4.573750,2.711215
1,-2.228665,1.182758,-4.573750,2.711215
2,-2.833490,1.623497,-4.894234,2.661258
3,-2.828129,1.620235,-4.977653,2.720906
4,-2.220202,1.178041,-5.085864,2.798276
...,...,...,...,...
52765702,32.252903,-53.570892,13.406327,-30.804071
52765703,32.741895,-53.426798,13.793980,-30.919076
52765704,32.978934,-53.348597,13.984154,-30.982821
52765705,31.460226,-51.603020,13.050610,-29.900243


In [12]:
dFrame_ = dFrame.drop_duplicates()

In [14]:
dFrame_

,0,1,2,3
0,-2.228665,1.182758,-4.573750,2.711215
2,-2.833490,1.623497,-4.894234,2.661258
3,-2.828129,1.620235,-4.977653,2.720906
4,-2.220202,1.178041,-5.085864,2.798276
5,-2.822828,1.617012,-5.219218,2.893616
...,...,...,...,...
52765141,43.894444,-77.166838,16.854947,-44.909684
52765181,38.540671,-68.536980,14.543449,-39.798988
52765230,44.979638,-71.919666,19.323364,-42.302775
52765238,47.306186,-75.259795,20.558415,-44.245095


In [15]:
array_crop_image = np.array(dFrame_) 
array_crop_image = 1.0/255*array_crop_image
array_crop_image.shape

(5072759, 4)

In [16]:
X_ = array_crop_image[:,0:2]
Y_ = array_crop_image[:,2:4]
# X_ = X_[:,1:]
# Y_ = Y_[:,1:]

# KERAS

In [17]:
# модель 1

model = Sequential()
model.add(Dense(100, input_dim=2, activation='sigmoid'))
model.add(ReLU(1.0))
model.add(Dense(50, activation='sigmoid'))
model.add(ReLU(1.0))
model.add(Dense(25, activation='softmax'))

#IMPORTANT PART
model.add(Dense(2, activation='linear'))

model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])


Metal device set to: Apple M1


2022-07-31 13:03:42.536437: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-31 13:03:42.536829: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [72]:
# модель 2

model = Sequential()
model.add(Dense(20, input_dim=2, activation='sigmoid'))
model.add(ReLU(1.0))
model.add(Dense(10, activation='sigmoid'))
model.add(ReLU(1.0))
model.add(Dense(5, activation='softmax'))

#IMPORTANT PART
model.add(Dense(2, activation='linear'))

model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])

In [19]:
model.fit(X_,Y_,batch_size=256, epochs=15, validation_split=0.2)

Epoch 1/15


2022-07-31 13:04:33.281242: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-31 13:04:33.405117: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


15853/15853 [==============================] - ETA: 0s - loss: 0.0013 - accuracy: 0.9253

2022-07-31 13:05:51.949064: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


15853/15853 [==============================] - 89s 6ms/step - loss: 0.0013 - accuracy: 0.9253 - val_loss: 0.0016 - val_accuracy: 0.8793
Epoch 2/15
15853/15853 [==============================] - 89s 6ms/step - loss: 0.0013 - accuracy: 0.9251 - val_loss: 0.0016 - val_accuracy: 0.8800
Epoch 3/15
15853/15853 [==============================] - 89s 6ms/step - loss: 0.0013 - accuracy: 0.9250 - val_loss: 0.0017 - val_accuracy: 0.8819
Epoch 4/15
15853/15853 [==============================] - 90s 6ms/step - loss: 0.0013 - accuracy: 0.9252 - val_loss: 0.0016 - val_accuracy: 0.8800
Epoch 5/15
15853/15853 [==============================] - 92s 6ms/step - loss: 0.0013 - accuracy: 0.9255 - val_loss: 0.0017 - val_accuracy: 0.8810
Epoch 6/15
15853/15853 [==============================] - 89s 6ms/step - loss: 0.0013 - accuracy: 0.9259 - val_loss: 0.0017 - val_accuracy: 0.8780
Epoch 7/15
15853/15853 [==============================] - 89s 6ms/step - loss: 0.0013 - accuracy: 0.9260 - val_loss: 0.0016 - val

In [27]:
model.save("/Users/alexeyvaganov/doc/files/color_matcher/temp/temp_keras_regression.h5")

# Предсказание

In [21]:
Obr = 105

In [20]:
file_name_massiv = []
#file_name_massiv.append("IMG_2837_m.jpeg")
#file_name_massiv.append("IMG_2825.jpeg")
file_name_massiv.append("IMG_3379.jpeg")
file_name_massiv.append("IMG_2915.jpeg")
file_name_massiv.append("IMG_2980.jpeg")
file_name_massiv.append("IMG_3107.jpeg")
file_name_massiv.append("IMG_3323.jpeg")
file_name_massiv.append("IMG_3336.jpeg")
file_name_massiv.append("IMG_2825.jpeg")
file_name_massiv.append("IMG_2769.jpeg")
file_name_massiv.append("IMG_2837.jpeg")
file_name_massiv.append("IMG_2847.jpeg")

In [22]:
def add_testing_file(file_name):
    test_image_path = image_massiv + "/" + file_name
    img_test = Image.open(test_image_path)
    img_test = img_test.convert('RGB')
    img_test = rgb2lab(img_test)
    return np.array(img_test) 

In [23]:
arrive_testing_file = []
for name_test_file in file_name_massiv:
    arr_ = add_testing_file(name_test_file)
    arrive_testing_file.append([name_test_file, arr_])

In [24]:
def predict_image(array_full_img_test):
    new_img = np.zeros((array_full_img_test.shape[0], array_full_img_test.shape[1], 3))
    data_p = []
    h=0
    while h <= array_full_img_test.shape[0]-pixel:
        w=0
        while w <= array_full_img_test.shape[1]-pixel:
            data_p.append([array_full_img_test[h,w][1],array_full_img_test[h,w][2]])
            w+=pixel
        h+=pixel 
        
    x_ = np.array(data_p)
    x_ = 1.0/255*x_
    #x_ = x_.reshape(1,2)
    pred_y = model.predict(x_)
    
    h=0
    i=0
    while h <= array_full_img_test.shape[0]-pixel:
        w=0
        while w <= array_full_img_test.shape[1]-pixel:
            new_img[h,w] = [array_full_img_test[h,w][0],pred_y[i][0]*256,pred_y[i][1]*256]
            i+=1
            w+=pixel
        h+=pixel 

    return new_img

# Предсказание здесь

In [25]:
Obr += 1
model_name = image_massiv + "/" + str(Obr) + "_type" + ".h5"
print(model_name)
for m_image in arrive_testing_file:
    p_image = predict_image(m_image[1])
    image_name = image_massiv + "/" + str(Obr) + "_" + m_image[0]
    print(image_name)
    imsave(image_name, lab2rgb(p_image))
model.save(model_name)

/Users/alexeyvaganov/doc/files/color_matcher/temp/106_type.h5
   84/38400 [..............................] - ETA: 1:10

2022-07-31 16:29:16.799737: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


38400/38400 [==============================] - 62s 2ms/step


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


/Users/alexeyvaganov/doc/files/color_matcher/temp/106_IMG_3379.jpeg
38400/38400 [==============================] - 69s 2ms/step


...

/Users/alexeyvaganov/doc/files/color_matcher/temp/106_IMG_2769.jpeg
38400/38400 [==============================] - 60s 2ms/step


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


/Users/alexeyvaganov/doc/files/color_matcher/temp/106_IMG_2837.jpeg
38400/38400 [==============================] - 60s 2ms/step


/Users/alexeyvaganov/miniforge3/lib/python3.9/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 50 pixels
  return func(*args, **kwargs)
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


/Users/alexeyvaganov/doc/files/color_matcher/temp/106_IMG_2847.jpeg


# Старые

In [124]:
array_full_img_test = arrive_testing_file[0][1]
new_img = np.zeros((array_full_img_test.shape[0], array_full_img_test.shape[1], 3))
h=0
while h <= array_full_img_test.shape[0]-pixel:
    w=0
    while w <= array_full_img_test.shape[1]-pixel:
        new_img[h,w] = [array_full_img_test[h,w][0],array_full_img_test[h,w][1],array_full_img_test[h,w][2]]
        w+=pixel
    h+=pixel 
imsave("/Users/alexeyvaganov/doc/files/color_matcher/temp/82_IMG_2837_m22.jpeg", lab2rgb(new_img))
#(color.convert_colorspace(new_img, 'HSV', 'RGB')*255).astype(np.uint8)
#(color.convert_colorspace(lab2rgb(new_img), 'HSV', 'RGB')*255).astype(np.uint8)

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [81]:
file_name = "IMG_2825.jpeg"
test_image_path = image_massiv + "/" + file_name
img_test = Image.open(test_image_path)
img_test = img_test.convert('RGB')
#array_img_test.append(img_test) 
img_test = rgb2lab(img_test)
array_full_img_test = np.array(img_test) 
array_full_img_test.shape

(1280, 960, 3)

In [46]:
model.save(image_massiv + "/" + str(Obr) + "type_.h5")

In [18]:
model = load_model("/Users/alexeyvaganov/doc/files/color_matcher/temp/101_type.h5")